Connected to base (Python 3.13.2)

In [1]:
import os
import requests
import time
import pandas as pd

# read secrets from environment
API_KEY = os.environ.get('API_KEY')
API_URL = os.environ.get('API_URL')

if not API_KEY or not API_URL:
    raise ValueError("Missing required tokens in environment variables.")

headers = {
    "Authorization": API_KEY
}

def get_timestamps_past_7_days():
    # Current time (end timestamp) in milliseconds
    end_ts = int(time.time() * 1000)
    # Start timestamp = 7 days ago in milliseconds
    start_ts = end_ts - (7 * 24 * 60 * 60 * 1000)
    return start_ts, end_ts

def fetch_telemetry():
    start_ts, end_ts = get_timestamps_past_7_days()
    
    params = {
        "keys": "temperature,electrical_conductivity,ph",
        "startTs": str(start_ts),
        "endTs": str(end_ts),
        "interval": "10800000",  # 3-hour interval
        "agg": "AVG",
        "orderBy": "ASC"
    }
    
    response = requests.get(API_URL, headers=headers, params=params)
    if response.status_code == 200:
        print("Response received from Edenic API: data")
        data = response.json()
        return data
    else:
        print(f"Failed to fetch telemetry data: HTTP {response.status_code}")
        print(response.text)
        return None

if __name__ == "__main__":
    data = fetch_telemetry()

Response received from Edenic API: data


In [24]:
# Process and export each parameter
for param_name, param_data in data.items():
    # Create DataFrame
    df = pd.DataFrame(param_data)
    
    # Convert timestamp from milliseconds to datetime
    df['ts'] = pd.to_datetime(df['ts'], unit='ms')
    
    # Convert value to float (except for pH which might need special handling)
    df['value'] = df['value'].astype(float)
    
    # Rename columns
    df = df.rename(columns={
        'ts': '',
        'value': param_name
    })

    # Export to CSV
    filename = f"edenic_{param_name}.csv"
    df.to_csv(filename, index=False)
    print(f"Exported {filename} with {len(df)} records")
    
    # Show sample of exported data
    print(f"\nSample of {filename}:")
    print(df.head(2), "\n" + "-"*50 + "\n")

Exported edenic_temperature.csv with 56 records

Sample of edenic_temperature.csv:
                           temperature
0 2025-07-19 08:59:14.737    31.993258
1 2025-07-19 11:59:14.737    32.083240 
--------------------------------------------------

Exported edenic_electrical_conductivity.csv with 56 records

Sample of edenic_electrical_conductivity.csv:
                           electrical_conductivity
0 2025-07-19 08:59:14.737                 4.930000
1 2025-07-19 11:59:14.737                 4.927727 
--------------------------------------------------

Exported edenic_ph.csv with 56 records

Sample of edenic_ph.csv:
                                 ph
0 2025-07-19 08:59:14.737  9.354719
1 2025-07-19 11:59:14.737  9.355642 
--------------------------------------------------

